### Helper

In [1]:
from manim import *
import numpy as np
from manim_slides import Slide

#### Preamble

In [2]:
# Preamble
preamble_path = "/home/ronakr/Documents/inplaceoracles/Inverting-a-Permutation-with-an-In-Place-Oracle/preamble.tex"
with open(preamble_path, 'r') as file:
    preamble = file.read()
myTemplate = TexTemplate()
myTemplate.add_to_preamble(preamble)

TexTemplate(_body='', tex_compiler='latex', description='', output_format='.dvi', documentclass='\\documentclass[preview]{standalone}', preamble='\\usepackage[english]{babel}\n\\usepackage{amsmath}\n\\usepackage{amssymb}\n\\usepackage[utf8]{inputenc}\n\n\\usepackage[letterpaper,margin=1in]{geometry}\n\\usepackage[OT1]{fontenc}\n\n\\usepackage{amsmath,amssymb,amsthm,amsfonts,latexsym,bbm,xspace,thm-restate}\n\\usepackage{graphicx,float,mathtools,braket,slantsc}\n\\usepackage{enumitem,booktabs,forest,mathdots,soul}\n\\usepackage[useregional]{datetime2}\n\\DTMusemodule{english}{en-US}\n\n\\usepackage{qcircuit,tikz}\n\n\\usepackage[colorlinks,citecolor=blue,,linkcolor=magenta,bookmarks=true]{hyperref}\n\\usepackage[capitalise]{cleveref}\n\n\\usepackage{multicol,array}\n\\usepackage{lipsum,framed}\n\n\\newtheorem{theorem}{Theorem}\n\\newtheorem*{theorem*}{Theorem}\n\\newtheorem{lemma}[theorem]{Lemma}\n\\newtheorem{claim}[theorem]{Claim}\n\\newtheorem{proposition}[theorem]{Proposition}\n\\ne

### Sanity Check

In [ ]:
%%manim -ql SanityCheck

class SanityCheck(Scene):
    def construct(self):
        s0 = MathTex(r"a")
        s1 = MathTex(r"\frac{a}{b}")
        s2 = MathTex(r"\frac{b}{a}")
        tms0 = TransformMatchingShapes(s0,s1, key_map={-6351953992051140758:8757917165807693890})
        tms = TransformMatchingShapes(s1,s2)#, key_map={-6351953992051140758:8757917165807693890})
        # s1a = tms.get_mobject_key(tms.get_mobject_parts(s1)[0])
        # s2l = tms.get_mobject_key(tms.get_mobject_parts(s2)[1])
        # tms.key_map = {s1a:s2l}
        print(tms0.get_mobject_parts(s0))
        print(tms0.get_mobject_parts(s1))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s0)[0]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[0]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[1]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[2]))

        print(tms.get_mobject_parts(s1))
        print(tms.get_mobject_parts(s2))

        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[0]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[1]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[2]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[0]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[1]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[2]))
        self.play(tms0)
        self.play(tms)

### Query Complexity (motivation)
- What is a problem? 
  - Input: some f, Output: some property of f. 
  - Egs: 
    - (1) Given: N-bit integer. Output: is it nonzero? (show arrow to "integer", saying f : [N] -> {0,1}, transform) 
      (show after (2)) Given: function f : [N] -> {0,1}. Output: is there an x such that f(x) = 1?
      mention this is "searching for a 1"
    - (2) Given: a shuffled deck of cards labeled {1,...,N}. Output: location of the "1" card.
      Given permutation f: [N] -> [N]. Output: x such that f(x) = 1.
- "In Complexity theory we often care about how much time the best algorithm to solve a problem would take as the size of the input (N) increases"
- "That's really hard to lower bound. The best technique we have is query complexity."
- In query cxty, we imagine the input is behind a paywall, and we need to pay an oracle every time we want to access bits of it. (oracle pic)
  - I will tell the oracle x and the oracle will tell me f(x), each request is called a query
  <!-- the label of the xth card in the deck. (exchange) -->
  - It's fairly intuitive that, for these specific problems above, N queries are necessary classically, (implying an Omega(N) lower bound on time?).
  - How do we do quantum query complexity? 
    - Our first thought might be |x> -> |f(x)>. 
      - Violates QM if f is not reversible. 
    - Solution: |x>|a> -> |x>|a + f(x)>. -self inverse -equiv to phase oracles
    - Okay, but what if f is a permutation? 
    - Then |x> -> |f(x)> is fine, called in-place queries.

### Key Question of this talk
- Given a permutation f, how do XOR queries and in-place queries to f compare?
  
  <!-- however, the ability to make multiple queries in superposition lets us find "1" with O(sqrt(N)) queries, through what's called Grover's algorithm
  - Understanding this alg is crucial to understanding our alg, so let's briefly review -->

### Table of comparisons
- If we have access to both f and f^{-1}, then both models can simulate the other
- Omega(sqrt(N)) XOR queries required to exactly simulate an in-place query
  - In this paper: Omega(sqrt(N)) in-place to XOR
- IndexErasure (state conversion) O(sqrt(N)) vs O(1)
  - states of the form |x>|f(x)> -> |0>|f(x)>
- SetComp (promise decision problem) O(N^{1/7}) vs O(1)
  - venn diagram highlighting: two large sets are the same or have large symmetric difference
- Unstructured search (search problem) O(sqrt(N)) vs conjectured Omega(N). (cross out conj, write ours) In this paper: O(sqrt(N)).
  - N shuffled cards, locate "1"
- Trashy Simon's (promise search problem) O(log(N)) vs conjectured Omega(sqrt(N)) ### TODO: verify sqrt(N) correct!

### Grover's
- Our problem: Given a shuffled deck of cards labeled [N], find 1. 
- (Show 12345 example truth table with f, circle inv of 1) 
- (transform problem) given a permutation f : [N] -> [N], find x^* := f^{-1}(1).
- (disappear truth table)
- In Grover's algorithm, we start off with a uniform superposition over all x in [N] (bar chart)
- Repeatedly alternate between the following two operations:
  - Query, which negates the amplitude on x^* (flip)
  - Grover's Diffusion operator, which flips all amplitudes about their average (draw line, flip)
  - (quickly evolve a few times until x^* peaks) after ~sqrt N iterations of this, we will have made ~sqrt N queries and the probability of seeing x^* when we measure.

### Where Grover's fails
- key step is being able to take |x> -> (-1)^{f(x) = 1} |x>
- |x>|0> -> |x>|f(x)> -> (-1)^{f(x) = 1}|x>|f(x)> (->) |x>|0>  <- function erasure is hard
- Without being able to erase f, interference fails


### Kashefi
- 
- Key fact: perm inv LBs for both Sf [Ambainis] and Pf [FK18]

### Kashefi Pfs (3 slides)
- Inverting circuits
- Kashefi arg (|x>|a> -[Sf]- |x>|f(x)> in corner so we can flip it to drive home that Sf inv = Sf)
- Our arg (|x> -[Pf]- |f(x)> in corner so we can flip it to drive home that Pf inv = P{f_inv})
- => function erasure is hard (analog of index erasure)

### Sf Pf box
- Grovers + Kashefi => O(sqrt(N)) XOR to in-place
- Our Alg + our arg => O(sqrt(N)) in-place to XOR

### Summary
- Unstructured Search takes Theta(sqrt(N)) queries (in-place or XOR) (algorithm)
- Function Erasure takes O(1) XOR Queries and O(sqrt(N)) in-place queries (kashefi)
- More generally, Sf Pf Sfinv Pfinv all req sqrt(N) queries to simulate each other
- Future work: decision problem separations
  - candidates: simon, emb perm inv


### TitleSlide

In [ ]:
%%manim -qm TitleSlide1

class TitleSlide1(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        title = Text("Quantum Search with In-Place Queries")
        subtitle = Text("Blake Holman, Ronak Ramachandran, and Justin Yirka", t2w={'Ronak Ramachandran':BOLD}).scale(0.5)
        subtitle.next_to(title, DOWN, buff=0.8)
        self.play(Add(title), Write(subtitle))
        self.next_slide()

### XORvsInPlace

In [ ]:
%%manim -ql XORvsInPlace

class XORvsInPlace(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        
        
        xor_bullets = [
            MathTex(r"\ket{x}\ket{a} \rightarrow \ket{x}\ket{f(x)}", tex_template=myTemplate),
            Text("Any f"),
            Text("Can erase f(x)"),
            Text("Self-inverse"),
            Text("Phase version")
        ]
        inplace_bullets = [
            MathTex(r"\ket{x} \rightarrow \ket{f(x)}", tex_template=myTemplate),
            Text("Bijections"),
            Text("Can erase x")
        ]
        
        xor_group = VGroup(*xor_bullets).arrange(DOWN)
        inplace_group = VGroup(*inplace_bullets).arrange(DOWN)

        tchart = MobjectTable(
            [[xor_group, inplace_group]],
            col_labels=[Text("XOR"), Text("In-Place")],
            arrange_in_grid_config={"row_alignments":'uu'})

        self.play(
            Write(tchart.get_horizontal_lines()), 
            Write(tchart.get_vertical_lines()),
            Write(tchart.get_col_labels()))
        self.next_slide()

        for i in range(len(xor_bullets)):
            self.play(Write(xor_bullets[i]))
            self.next_slide()

            if i < len(inplace_bullets):
                self.play(Write(inplace_bullets[i]))
                self.next_slide()

### Inverting Circuits

In [13]:
def _flip(x,y,z,t):
    return (np.cos(t*np.pi)*x, 0.5*x*np.sin(t*np.pi)+y, z)

def Flip(mobject=None, *vargs, **kwargs):
    return Homotopy(_flip, mobject, *vargs, **kwargs)

In [3]:
def FlipTransform(self, mobject_from=None, mobject_to=None):
    self.play(Homotopy(lambda x,y,z,t : (np.cos(t*np.pi/2)*x, y + 0.2*x*np.sin(t*np.pi/2), z), mobject_from, rate_func=rush_into, run_time=0.8))
    self.play(Homotopy(lambda x,y,z,t : (np.sin(t*np.pi/2)*x, y - 0.2*x*np.cos(t*np.pi/2), z), mobject_to, rate_func=rush_from, run_time=0.8))

In [10]:
%%manim -ql InvertingCircuits1

def brace_ckt(ckt, left, right):
    bL = Brace(ckt, LEFT)
    bLt = bL.get_tex(left)
    bR = Brace(ckt, RIGHT)
    bRt = bR.get_tex(right)
    return VGroup(bL, bLt, bR, bRt)

class InvertingCircuits1(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        abcde_tex = r'''\Qcircuit @C=0.7em @R=0.7em @!R {
            & \gate{A} & \multigate{1}{C} & \gate{D} & \qw \\
            & \gate{B} & \ghost{C} & \gate{E} & \qw
        }'''
        abcde_ckt = MathTex(abcde_tex, tex_template=myTemplate)
        abcde_braces = brace_ckt(abcde_ckt, r"|\psi\rangle \rightarrow", r"\rightarrow \mathcal{C}|\psi\rangle")
        abcde = VGroup(abcde_ckt, abcde_braces)

        self.play(Write(abcde))
        self.next_slide()

        inv_abcde_tex = r'''\Qcircuit @C=0.7em @R=0.7em @!R {
            & \gate{D^{-1}} & \multigate{1}{C^{-1}} & \gate{A^{-1}} & \qw \\
            & \gate{E^{-1}} & \ghost{C^{-1}} & \gate{B^{-1}} & \qw
        }'''
        inv_abcde_ckt = MathTex(inv_abcde_tex, tex_template=myTemplate)
        inv_abcde_braces = brace_ckt(inv_abcde_ckt, r"\mathcal{C} |\psi\rangle \rightarrow", r"\rightarrow |\psi\rangle")
        inv_abcde = VGroup(inv_abcde_ckt, inv_abcde_braces)

        FlipTransform(self, abcde, inv_abcde)
        self.next_slide()

        new_inv_abcde_braces = brace_ckt(inv_abcde_ckt, r"|\psi\rangle \rightarrow", r"\rightarrow \mathcal{C}^{-1} |\psi\rangle")
        self.play(Transform(inv_abcde_braces, new_inv_abcde_braces))
        # self.play(*[TransformMatchingShapes(inv_abcde_braces[i], new_inv_abcde_braces[i]) for i in (1,3)])
        self.next_slide()
        

Manim Community v0.19.0

[07/03/25 22:52:33] INFO     Animation 0 : Using cached data (hash :                           ]8;id=320095;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=195527;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1749642518_2614666737_223132457)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=235096;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=594161;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1842175414_1443166991_1603388940)                                                     

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=212251;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=264187;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1842175414_629272221_2375220172)                                                      

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=575684;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=704024;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1842175414_4088823500_2893859708)                                                     

...

                    INFO     Rendered InvertingCircuits1                                               ]8;id=288211;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=710737;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 7 animations                                                                   

[07/03/25 22:52:35] INFO     Generated 3 slides to                                                      ]8;id=917071;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=367397;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/InvertingCircuits1'                  

                    INFO     Generated 3 slides to                                                      ]8;id=923324;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=20779;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/InvertingCircuits1'                  

                    INFO     Slide 'InvertingCircuits1' configuration written in                        ]8;id=741528;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=9517;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/InvertingCircuits1.json'                   

                    INFO     Slide 'InvertingCircuits1' configuration written in                        ]8;id=791803;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=828914;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/InvertingCircuits1.json'                   

### Kashefi

In [32]:
%%manim -ql Kashefi

class Kashefi(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        
        ckt_outline = RoundedRectangle(corner_radius=0.25, height=4.0, width=8.0)
        oracle_tex = r'''\Qcircuit @C=0.7em @R=0.7em @!R { & \gate{P_f} & \qw }'''
        oracles = []
        oracles.append(MathTex(oracle_tex, tex_template=myTemplate).move_to(ckt_outline.get_center()).shift(0.8*LEFT))
        oracles.append(oracles[0].copy().shift(1.5*LEFT + UP))
        oracles.append(oracles[0].copy().shift(1.5*RIGHT + UP))
        oracles.append(oracles[0].copy().shift(3*RIGHT + DOWN))
        ckt = VGroup(ckt_outline, *oracles)
        
        self.play(FadeIn(ckt))
        self.play(ckt.animate.flip())

Manim Community v0.19.0

[07/03/25 23:09:02] INFO     Animation 0 : Using cached data (hash :                           ]8;id=803982;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=70403;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1749642518_3029414909_223132457)                                                      

[07/03/25 23:09:03] INFO     Animation 1 : Partial movie file written in                   ]8;id=339285;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=623870;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Kashefi/1842175414_1105763165_2023                         
                             860264.mp4'                                                                           

                    INFO     Combining to Movie file.                                      ]8;id=969760;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=337185;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=732874;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=14305;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/Kashefi.mp4'                                                                   
                                                                                                                   

                    INFO     Rendered Kashefi                                                          ]8;id=288923;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=740988;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   

[07/03/25 23:09:04] INFO     Generated 1 slides to                                                      ]8;id=827528;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=550216;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Kashefi'                             

[07/03/25 23:09:04] INFO     Generated 1 slides to                                                      ]8;id=224880;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=998533;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Kashefi'                             

                    INFO     Slide 'Kashefi' configuration written in                                   ]8;id=310337;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=207075;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Kashefi.json'                              

                    INFO     Slide 'Kashefi' configuration written in                                   ]8;id=762862;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=982706;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Kashefi.json'                              

### AlgCkt

In [ ]:
%%manim -ql AlgCircuit

class AlgCircuit(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        # qcircuit
        qcircuit = r'''
        \Qcircuit @C=.5em @R=0.5em @!R {
         & & & & & & \mbox{\hspace{1.8em}\textit{Mark}} & & & & & & & & \mbox{\textit{Shift}} & & & & & & & & \mbox{\textit{Diffuse the Difference}} \\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \ctrl{4}  & \qw       & \qw       & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \multigate{2}{\Ppi} & \ctrl{3}  & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \multigate{2}{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \ctrl{4}  & \qw       & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ghost{\Ppi} & \qw       & \ctrl{2}  & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \ghost{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \rstick{\ket{\psi_{t}}_\regA} \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \ctrl{4}  & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ghost{\Ppi} & \qw       & \qw       & \ctrl{1}  & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \ghost{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \push{\rule{0em}{1em}}
    \inputgroupv{2}{4}{0.5em}{2em}{\ket{\psi_{t-1}}_\regA \hspace{1.3em}} \\
    	& \lstick{\ket{0}_\regC \Big\{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \qw       & \qw       & \targ     & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ctrlo{-1} & \ctrlo{1} & \ctrlo{2} & \ctrlo{3} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \gate{H}  & \ctrl{-1} & \gate{H}  & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \meter    & \cw       & \rstick{\text{\textit{Measure} }\regC} \\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \targ     & \qw       & \qw       & \multigate{2}{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \targ     & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \targ     & \qw       & \ghost{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \targ     & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \rstick{\ket{0^n}_\regB} \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \targ     & \ghost{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \qw       & \targ     & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}}
    \inputgroupv{6}{8}{0.5em}{2em}{\ket{0^n}_\regB \hspace{0.5em}}
    \gategroup{2}{29}{4}{29}{0em}{\}}
    \gategroup{6}{21}{8}{21}{0em}{\}}
    \gategroup{2}{4}{8}{10}{0.7em}{--}
    \gategroup{2}{13}{8}{18}{0.7em}{--}
    \gategroup{2}{21}{5}{25}{0.7em}{--}\\
    }
        '''
        label = MathTex(qcircuit, tex_template=myTemplate).scale(0.5)
        self.play(Write(label))
        self.wait()
        self.next_slide()

### StepThruOurAlgBar

In [ ]:
def get_HRY_amps(t, N=10):
    a_t = np.sin((t+1)*np.arctan(1/np.sqrt(N-1)))
    b_t = np.cos((t+1)*np.arctan(1/np.sqrt(N-1)))/np.sqrt(N-1)
    assert(np.isclose(a_t**2 + (N-1)*b_t**2, 1))
    return a_t, b_t

In [ ]:
%%manim -ql StepThruOurAlgBar

class StepThruOurAlgBar(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        N = 10
        T = 21

        chart = BarChart(
            [1/np.sqrt(N)]*N,
            y_range=[-1, 1, 1],
            y_axis_config={"font_size": 24},
        )

        self.play(DrawBorderThenFill(chart))
        self.next_slide()

        for t in range(1,T):
            a_t,b_t = get_HRY_amps(t,N)
            values = [a_t] + [b_t]*(N-1)
            self.play(chart.animate.change_bar_values(values))
            self.next_slide()

        # charts = []
        # # labels = []
        # for t in range(T):
        #     a_t,b_t = get_amps(t,N)
        #     values = [a_t] + [b_t]*(N-1)

        #     charts.append(BarChart(
        #         values,
        #         y_range=[-1, 1, 1],
        #         y_axis_config={"font_size": 24},
        #     ))

        #     # labels.append(charts[t].get_bar_labels(font_size=24))

        # self.play(DrawBorderThenFill(charts[0])) #, Write(labels[0]))
        # self.wait()
        
        # for t in range(1,T):
        #     self.play(Transform(charts[t-1],charts[t])) #,Transform(labels[t-1],labels[t]))
        #     self.wait()
